In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from scipy import stats
linkedin = pd.read_csv("linkedin_company.csv",low_memory=False)
linkedin=linkedin.drop(['link','date_added','date_updated','description','website','entity_id','cusip','isin'],axis=1)
linkedin.head()

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,industry
0,58329,2015-09-14,Goldman Sachs,552254,38124,Investment Banking
1,58329,2015-09-15,Goldman Sachs,552862,38141,Investment Banking
2,58363,2015-09-16,United Technologies,59157,14982,Aviation & Aerospace
3,58366,2015-09-16,Novo Nordisk,336175,26448,Pharmaceuticals
4,58371,2015-09-16,"Lowe's Companies, Inc.",134255,62574,Retail


In [3]:
Now=linkedin[linkedin['as_of_date']>='2018-01-01'].drop_duplicates(subset='dataset_id',keep='last')
Past=linkedin[linkedin['as_of_date']<'2016-01-01'].drop_duplicates(subset='dataset_id',keep='first')
Past=Past.sort_values('followers_count',ascending=False).reset_index(drop=True)
Past.loc[:,'rank_2015']=Past.index
Top10_Now=Now.sort_values('followers_count',ascending=False).head(10)
Top10=pd.merge(Top10_Now,Past,how='left',on='dataset_id')
Top10.followers_count_y.fillna(0,inplace=True)
Top10.loc[:,'rank_increase']=Top10.rank_2015-Top10.index
Top10.rank_increase.fillna('NA',inplace=True)
trace1=go.Bar(
            y=[Top10.company_name_x,Top10.rank_increase],
            x=Top10.followers_count_x,
            name='2018',
            orientation='h'
            )

trace2=go.Bar(
            y=[Top10.company_name_x,Top10.rank_increase],
            x=Top10.followers_count_y,
            name='2015',
            orientation='h'
            )
layout=go.Layout(
                barmode='group',
                title='2018 TOP 10 followed company at Linkedin',
                yaxis=dict(autorange='reversed'),
                xaxis=dict(title='Followers Count'),
                )

iplot(go.Figure(data=[trace1,trace2],layout=layout))